# import packages

In [1]:
import pandas as pd
import dask.dataframe as dd
import numpy as np
import json
from time import time

from collections import Counter

from konlpy.corpus import kolaw
from konlpy.tag import Hannanum
from konlpy.utils import concordance, pprint
import matplotlib.pyplot as plt
from matplotlib import pyplot

from konlpy.tag import *
from gensim.models.word2vec import Word2Vec

km = Komoran()
tw = Twitter()
han = Hannanum()
kkma = Kkma()

def draw_zipf(count_list, filename, color='blue', marker='o'):
    sorted_list = sorted(count_list, reverse=True)
    plt.plot(sorted_list, color=color, marker=marker)
    plt.xscale('log')
    plt.yscale('log')
    plt.show()
    #pyplot.savefig(filename)


C:\Anaconda2\envs\py36\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


# 나무위키

In [2]:
with open('C:/Users/boogi/Downloads/namuwiki_20180326.json') as f:
    data = json.load(f)

In [3]:
data[0]

{'contributors': ['namubot', 'R:hoon12560'],
 'namespace': '0',
 'text': '#redirect 느낌표\n',
 'title': '!'}

In [4]:
'''
import dask.bag as db
js = db.read_text('C:/Users/boogi/Downloads/namuwiki_1.json').map(json.loads)
js.take(1)
del js
'''

"\nimport dask.bag as db\njs = db.read_text('C:/Users/boogi/Downloads/namuwiki_1.json').map(json.loads)\njs.take(1)\ndel js\n"

In [5]:
len(data)

565985

In [3]:
keys = ['카지노', '호텔', '도박장']
minimal = ''
for i in range(len(data)):
    txt = '%s' % data[i]
    if any([True if key in txt else False for key in keys]):
        for item in txt.split('\\n'):
            if any([True if key in item else False for key in keys]):
                for jtem in item.split('.'):
                    if any([True if key in jtem else False for key in keys]):
                        minimal += jtem + '. '

In [4]:
len(minimal.split('.'))

25508

In [5]:
del data

In [6]:
minimal = minimal.replace(']', '').replace('[', '').replace('*', '').replace('|', '').replace("\\'", "").replace('~', '')

# RNN test

In [7]:
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, Dropout
from keras.optimizers import *
from keras.utils import np_utils
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer

from nltk.tokenize import sent_tokenize
#from konlpy.corpus import kolaw
#from konlpy.tag import Twitter

Using TensorFlow backend.


In [8]:
doc0 = [" ".join(["".join(w) for w, t in tw.pos(s) 
                  if (t not in ["Foreign"])]) for s in sent_tokenize(minimal) if (s.count(' ') > 4) & (s.count(' ') < 13)]
doc0 = [item for item in doc0 if item]

In [9]:
len(doc0)

9516

In [34]:
doc0 = [" ".join(["".join(w) for w, t in tw.pos(s) 
                  if (t not in ["Foreign"])]) for s in sent_tokenize(minimal)]

In [35]:
len(doc0)

26296

In [11]:
'''doc0 = [" ".join(["".join(w) for w, t in tw.pos(s) 
                  if t not in ['Number',"Foreign"]]) for s in sent_tokenize(minimal)]'''

In [10]:
tokenizer = Tokenizer()
doc_temp = doc0[:5000]
tokenizer.fit_on_texts(doc_temp)
doc = [l for l in tokenizer.texts_to_sequences(doc_temp) if len(l) > 1]

maxlen = max([len(x) - 1 for x in doc])
vocab_size = len(tokenizer.word_index) + 1

In [11]:
maxlen, vocab_size

(52, 13541)

In [12]:
def generate_data(X, maxlen, vocab_size):
    for sentence in X: 
        inputs = []
        targets = []
        for i in range(1, len(sentence)):
            inputs.append(sentence[0:i])
            targets.append(sentence[i])
        y = np_utils.to_categorical(targets, vocab_size)
        inputs_sequence = sequence.pad_sequences(inputs, maxlen=maxlen)
        yield (inputs_sequence, y)

In [13]:
X = []
Y = []
for x, y in generate_data(doc, maxlen, vocab_size):
    X.append(x)
    Y.append(y)

X = np.concatenate(X)
Y = np.concatenate(Y)

print(X.shape, Y.shape)

(78151, 52) (78151, 13541)


In [14]:
model = Sequential()
model.add(Embedding(vocab_size, 100, input_length=maxlen))
model.add(LSTM(100, return_sequences=False))
model.add(Dropout(0.5))
model.add(Dense(vocab_size, activation='softmax'))

In [15]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 52, 100)           1354100   
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dropout_1 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 13541)             1367641   
Total params: 2,802,141
Trainable params: 2,802,141
Non-trainable params: 0
_________________________________________________________________


In [16]:
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(), metrics=["accuracy"])

In [ ]:
%%time
hist = model.fit(X, Y, epochs=500, batch_size=800, verbose=2)

Epoch 1/500
19s - loss: 3.7149 - acc: 0.5484
Epoch 2/500
19s - loss: 3.7091 - acc: 0.5494
Epoch 3/500
19s - loss: 3.7067 - acc: 0.5486
Epoch 4/500
19s - loss: 3.7053 - acc: 0.5503
Epoch 5/500
18s - loss: 3.7087 - acc: 0.5483
Epoch 6/500
18s - loss: 3.7080 - acc: 0.5475
Epoch 7/500
19s - loss: 3.7010 - acc: 0.5492
Epoch 8/500
19s - loss: 3.6993 - acc: 0.5499
Epoch 9/500
19s - loss: 3.7070 - acc: 0.5485
Epoch 10/500
19s - loss: 3.6980 - acc: 0.5510
Epoch 11/500
19s - loss: 3.6959 - acc: 0.5507
Epoch 12/500
19s - loss: 3.6946 - acc: 0.5511
Epoch 13/500
19s - loss: 3.6952 - acc: 0.5512
Epoch 14/500
19s - loss: 3.6913 - acc: 0.5506
Epoch 15/500
19s - loss: 3.6934 - acc: 0.5524
Epoch 16/500
19s - loss: 3.6884 - acc: 0.5521
Epoch 17/500
19s - loss: 3.6895 - acc: 0.5525
Epoch 18/500
19s - loss: 3.6886 - acc: 0.5516
Epoch 19/500
19s - loss: 3.6891 - acc: 0.5521
Epoch 20/500
19s - loss: 3.6857 - acc: 0.5518
Epoch 21/500
18s - loss: 3.6840 - acc: 0.5529
Epoch 22/500
19s - loss: 3.6847 - acc: 0.55

19s - loss: 3.5390 - acc: 0.5744
Epoch 179/500
19s - loss: 3.5416 - acc: 0.5745
Epoch 180/500
19s - loss: 3.5338 - acc: 0.5762
Epoch 181/500
19s - loss: 3.5391 - acc: 0.5761
Epoch 182/500
19s - loss: 3.5342 - acc: 0.5763
Epoch 183/500
18s - loss: 3.5398 - acc: 0.5753
Epoch 184/500
19s - loss: 3.5378 - acc: 0.5748
Epoch 185/500
19s - loss: 3.5380 - acc: 0.5742
Epoch 186/500
19s - loss: 3.5315 - acc: 0.5753
Epoch 187/500
19s - loss: 3.5348 - acc: 0.5752
Epoch 188/500
19s - loss: 3.5380 - acc: 0.5752
Epoch 189/500
19s - loss: 3.5327 - acc: 0.5766
Epoch 190/500
19s - loss: 3.5343 - acc: 0.5750
Epoch 191/500
19s - loss: 3.5295 - acc: 0.5775
Epoch 192/500
18s - loss: 3.5332 - acc: 0.5746
Epoch 193/500
19s - loss: 3.5325 - acc: 0.5759
Epoch 194/500
19s - loss: 3.5323 - acc: 0.5738
Epoch 195/500
18s - loss: 3.5258 - acc: 0.5767
Epoch 196/500
18s - loss: 3.5294 - acc: 0.5758
Epoch 197/500
19s - loss: 3.5301 - acc: 0.5762
Epoch 198/500
20s - loss: 3.5261 - acc: 0.5761
Epoch 199/500
21s - loss: 3

18s - loss: 3.4573 - acc: 0.5873
Epoch 354/500
19s - loss: 3.4508 - acc: 0.5871
Epoch 355/500
19s - loss: 3.4533 - acc: 0.5876
Epoch 356/500
19s - loss: 3.4501 - acc: 0.5878
Epoch 357/500
19s - loss: 3.4496 - acc: 0.5883
Epoch 358/500
19s - loss: 3.4507 - acc: 0.5895
Epoch 359/500
19s - loss: 3.4508 - acc: 0.5890
Epoch 360/500
19s - loss: 3.4509 - acc: 0.5880
Epoch 361/500
19s - loss: 3.4510 - acc: 0.5888
Epoch 362/500
19s - loss: 3.4485 - acc: 0.5886
Epoch 363/500
19s - loss: 3.4508 - acc: 0.5874
Epoch 364/500
19s - loss: 3.4464 - acc: 0.5894
Epoch 365/500
19s - loss: 3.4465 - acc: 0.5893
Epoch 366/500
19s - loss: 3.4473 - acc: 0.5890
Epoch 367/500
19s - loss: 3.4455 - acc: 0.5893
Epoch 368/500
19s - loss: 3.4453 - acc: 0.5892
Epoch 369/500
19s - loss: 3.4444 - acc: 0.5892
Epoch 370/500
19s - loss: 3.4482 - acc: 0.5880
Epoch 371/500
19s - loss: 3.4407 - acc: 0.5906
Epoch 372/500
19s - loss: 3.4408 - acc: 0.5897
Epoch 373/500
19s - loss: 3.4436 - acc: 0.5897
Epoch 374/500
18s - loss: 3

In [ ]:
model.save("rnn_text_gen.hdf5")

In [ ]:
plt.plot(hist.history['acc'])
plt.show()

In [ ]:
def generate(w, n, seed=None):
    
    if seed is not None:
        np.random.seed(seed)
    
    def _predict_word(sent):
        x = sequence.pad_sequences([[tokenizer.word_index[w] for w in sent]], maxlen=maxlen)
        p = model.predict(x)[0]
        logp = np.log(p)
        p = np.exp(logp) / np.sum(np.exp(logp))
        p = p[1:]
        return np.random.choice(list(tokenizer.word_index.keys()), p=p)

    sent = [w]
    for i in range(n):
        w = _predict_word(sent)
        sent.append(w)
        
    return " ".join(sent)

In [76]:
doc0[10] = word_list

'샌프란시스코 Fairmont Today'

In [60]:
def predict_word(i, n=1):
    x = sequence.pad_sequences([[tokenizer.word_index[w] for w in word_list[:i]]], maxlen=maxlen)
    p = model.predict(x)[0]
    idx = np.flip(np.argsort(p), 0)
    for j in idx[:n]:
        print('"', " ".join(word_list[:i]), '"', reverse_word_map[j], " (p={:4.2f}%)".format(100 * p[j]))

In [61]:
predict_word(1)

NameError: name 'word_list' is not defined

In [ ]:
# 국문
def korean_most_common(c, n, pos=None):
    bg = bigram
    if pos is None:
        return bigram[tokenize(c)[0]].most_common(n)
    else:
        return bigram["/".join([c, pos])].most_common(n)
    
def korean_generate_sentence(seed=None, debug=False):
    if seed is not None:
        np.random.seed(seed)
    c = "SS"
    sentence = []
    while True:
        if c not in bigram:
            break
        words, probs = zip(*[(k, v) for k, v in bigram[c].items()])
        idx = np.argmax(np.random.multinomial(1, probs, (1,)))
        w = words[idx]
        
        if w == "SE":
            break            
        
        w2 = w.split("/")[0]
        pos = w.split("/")[1]
        
        if c == "SS":
            sentence.append(w2.title())
        elif c in ["`", "\"", "'", "("]:
            sentence.append(w2)
        elif w2 in ["'", ".", ",", ")", ":", ";", "?"]:
            sentence.append(w2)
        elif pos in ["Josa", "Punctuation", "Suffix"]:
            sentence.append(w2)
        elif w in ["임/Noun", "것/Noun", "는걸/Noun", "릴때/Noun",
                   "되다/Verb", "이다/Verb", "하다/Verb", "이다/Adjective"]:
            sentence.append(w2)
        else:
            sentence.append(" " + w2)
        c = w
        
        if debug:
            print(w)
            
    return "".join(sentence)